#### Байесовская классификация спам/не спам

Импортируем библиотеки

In [21]:
import pandas as pd
import sklearn.feature_extraction.text as tx, sklearn.naive_bayes as nb, sklearn.linear_model as lm, sklearn.ensemble as en, sklearn.metrics as me, sklearn.model_selection as ms
from warnings import filterwarnings as fw
fw('ignore')

Узнаем вариант

In [22]:
ord('А')%3

2

Прочитаем корпус текстов

In [23]:
corpus = pd.read_csv('Email spam 2 .csv')
corpus

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


Проанализируем количество слов по классам:

In [31]:
cnt, cntS, cntH = 0, 0, 0
for i in corpus['text']:
    cnt += len(i.split(' '))
for i in corpus[corpus['label']=='spam']['text']:
    cntS += len(i.split(' '))
for i in corpus[corpus['label']=='ham']['text']:
    cntH += len(i.split(' '))
print('Всего строк:', corpus.shape[0], '\nИз них', 
    corpus['label'].value_counts()['spam'], 'spam и',corpus['label'].value_counts()['ham'], 'ham', 
    '\nВсего слов: ', cnt, '\nиз них', cntS, 'spam и', cntH, 'ham')

Всего строк: 5171 
Из них 1499 spam и 3672 ham 
Всего слов:  1083244 
из них 320863 spam и 762381 ham


In [32]:
xtrain, xtest, ytrain, ytest = ms.train_test_split(corpus['text'],corpus['label'], test_size=0.2)
print('Обучающая выборка:', xtrain.shape,'\nТестовая выборка', xtest.shape)

Обучающая выборка: (4136,) 
Тестовая выборка (1035,)


-- -- 
Используем все слова с учетом регистра:

In [33]:
cv = tx.CountVectorizer(lowercase=False)
cv.fit(corpus['text'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 50448 токенов


In [34]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98472   0.97123   0.97793       730
        spam    0.93333   0.96393   0.94839       305

    accuracy                        0.96908      1035
   macro avg    0.95903   0.96758   0.96316      1035
weighted avg    0.96958   0.96908   0.96922      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97934   0.97397   0.97665       730
        spam    0.93851   0.95082   0.94463       305

    accuracy                        0.96715      1035
   macro avg    0.95893   0.96240   0.96064      1035
weighted avg    0.96731   0.96715   0.96721      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99164   0.97534   0.98343       730
        spam    0.94322   0.98033   0.96141       305

    accuracy                        0.97681      1035
   macro avg    0.96743   0.97784   0.97242      1035
weighted avg 

-- -- 
Используем все слова, но в одном регистре:

In [35]:
cv = tx.CountVectorizer()
cv.fit(corpus['text'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 50447 токенов


In [36]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98472   0.97123   0.97793       730
        spam    0.93333   0.96393   0.94839       305

    accuracy                        0.96908      1035
   macro avg    0.95903   0.96758   0.96316      1035
weighted avg    0.96958   0.96908   0.96922      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97796   0.97260   0.97527       730
        spam    0.93528   0.94754   0.94137       305

    accuracy                        0.96522      1035
   macro avg    0.95662   0.96007   0.95832      1035
weighted avg    0.96538   0.96522   0.96528      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98889   0.97534   0.98207       730
        spam    0.94286   0.97377   0.95806       305

    accuracy                        0.97488      1035
   macro avg    0.96587   0.97456   0.97007      1035
weighted avg 

-- -- 
Найдем все стоп-слова, которые встречаются в большом количестве сообщений (более 500 , что около 9% всей выборки) и уберем их:

In [37]:
cv = tx.CountVectorizer(max_df = 500)
cv.fit(corpus['text'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
print(stop)

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 117 токенов
{'what', 'out', 'that', 'of', 'please', 'has', 'like', 'some', 'enron', 'us', 'on', 'do', 'they', 'at', 'gas', 'see', 'if', 'daren', 'email', 'it', 'all', 'com', 'message', 'about', 'let', 'questions', 'should', 're', 'only', 'new', '01', 'and', 'can', 'for', 'have', 'nom', 'with', 'just', 'thanks', '2000', 'so', 'which', 'information', 'when', 'been', 'hou', 'need', 'ect', 'get', 'into', 'by', 'but', 'know', 'was', 'there', 'my', 'as', 'our', 'xls', 'attached', 'will', 'meter', 'time', 'mmbtu', 'pm', '03', 'one', 'subject', 'am', 'cc', 'farmer', 'would', 'we', '000', '2001', 'the', '11', 'also', 'here', 'an', 'http', 'to', 'corp', 'forwarded', 'in', 'may', 'no', 'any', '12', 'these', 'file', 'price', 'you', 'now', 'hpl', 'are', 'your', 'this', 'month', 'sent', 'be', '30', '02', 'or', 'call', 'up', 'deal', '20', 'me', '10', 'volume', 'day', 'from', '00', 'is', 'more', 'not'}


In [38]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98212   0.97808   0.98010       730
        spam    0.94805   0.95738   0.95269       305

    accuracy                        0.97198      1035
   macro avg    0.96509   0.96773   0.96639      1035
weighted avg    0.97208   0.97198   0.97202      1035
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.98580   0.95068   0.96792       730
        spam    0.89124   0.96721   0.92767       305

    accuracy                        0.95556      1035
   macro avg    0.93852   0.95895   0.94780      1035
weighted avg    0.95793   0.95556   0.95606      1035
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98491   0.98356   0.98424       730
        spam    0.96078   0.96393   0.96236       305

    accuracy                        0.97778      1035
   macro avg    0.97285   0.97375   0.97330      1035
weighted avg 

Найдем все стоп-слова, которые встречаются в маленьком количестве сообщений (менее 10, что около 0.18% всей выборки), отфильтруем только те слова, которые характерны какому-то классу (разница в частотах по классам более, чем в 7 раза):

In [ ]:
cv = tx.CountVectorizer(min_df = 10)
cv.fit(corpus['text'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов отмечены как редкие.')
#print(stop) # не будем выводить список слов, так как он слишком большой

all_spam = corpus[corpus['label']=='spam'].groupby('label').agg({'text':' '.join}).to_string()
all_ham = corpus[corpus['label']=='ham'].groupby('label').agg({'text':' '.join}).to_string()
temp = set()
for i in stop:
    frH = all_ham.count(i + ' ')/cntH
    frS = all_spam.count(i + ' ')/cntS
    if(frS != 0 and frH != 0):
        if(not(frS/frH > 7 or frS/frH < 1/7)):
            temp.add(i)
stop = temp
print('Из них {0} удовлетворяют условиям.'.format(len(stop)))

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
print('Работаем с {0} - {1} = {2}'.format(len(cv.vocabulary_) + len(stop), len(stop), len(cv.vocabulary_)), 'токенами.')

In [ ]:
cv = tx.CountVectorizer(stop_words = stop)
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

In [ ]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Найдем все стоп-слова, которые встречаются в маленьком количестве сообщений (менее 10, что около 0.18% всей выборки), отфильтруем только те слова, которые не характерны какому-то классу (разница в частотах по классам менее, чем в 3 раза):

In [ ]:
cv = tx.CountVectorizer(min_df = 10)
cv.fit(corpus['text'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов отмечены как редкие.')
#print(stop) # не будем выводить список слов, так как он слишком большой

all_spam = corpus[corpus['label']=='spam'].groupby('label').agg({'text':' '.join}).to_string()
all_ham = corpus[corpus['label']=='ham'].groupby('label').agg({'text':' '.join}).to_string()
temp = set()
for i in stop:
    frH = all_ham.count(i + ' ')/cntH
    frS = all_spam.count(i + ' ')/cntS
    if(frS != 0 and frH != 0):
        if(not(frS/frH < 3 or frS/frH >1/3):
            temp.add(i)
stop = temp
print('Из них {0} удовлетворяют условиям.'.format(len(stop)))

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
print('Работаем с {0} - {1} = {2}'.format(len(cv.vocabulary_) + len(stop), len(stop), len(cv.vocabulary_)), 'токенами.')

In [ ]:
cv = tx.CountVectorizer(stop_words = stop)
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

In [ ]:
lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Используем встроенный список стоп-слов:

In [15]:
cv = tx.CountVectorizer(stop_words = 'english')
cv.fit(corpus['text'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

lr =  lm.LogisticRegression()
rfc = en.RandomForestClassifier()
cnb = nb.ComplementNB()

lr.fit(tr_xtrain,ytrain)
rfc.fit(tr_xtrain,ytrain)
cnb.fit(tr_xtrain,ytrain)

pred_lr = lr.predict(tr_xtest)
pred_rfc = rfc.predict(tr_xtest)
pred_cnb = cnb.predict(tr_xtest)

print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
    'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
    'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97465   1.00000   0.98716       961
        spam    1.00000   0.83766   0.91166       154

    accuracy                        0.97758      1115
   macro avg    0.98732   0.91883   0.94941      1115
weighted avg    0.97815   0.97758   0.97673      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97660   0.99896   0.98765       961
        spam    0.99242   0.85065   0.91608       154

    accuracy                        0.97848      1115
   macro avg    0.98451   0.92480   0.95187      1115
weighted avg    0.97879   0.97848   0.97777      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99552   0.92508   0.95901       961
        spam    0.67568   0.97403   0.79787       154

    accuracy                        0.93184      1115
   macro avg    0.83560   0.94955   0.87844      1115
weighted avg 